In [1]:
import pandas as pd
import requests
import numpy as np
from pandas import json_normalize
from pathlib import Path
from api_key import api_key

In [2]:
# Station codes as strings
station_code_list = ["BG001"]

In [3]:
# Headers with API key
headers = {
    "api_key": api_key
}

In [4]:
# Initialize an empty list to store the data for all stations
all_stations_data = []

# Number of iterations
num_iterations = 300

# Iterate through each station
for station in station_code_list:
    # Iterate over the specified number of times
    for i in range(num_iterations):
        # Parameters for the specific station
        url = f"https://api.dpird.wa.gov.au/v2/weather/stations/{station}/data?"
        params = {
            "startDateTime": "2023-07-01T00:00:00Z",
            "endDateTime": "2024-01-01T00:00:00Z",
            "offset": i * 1000,  # Use the current iteration index as the offset
            "limit": 1000,
        }

        # Make the request for each station
        response = requests.get(url, params=params, headers=headers)
        data = response.json()

        # Check if the 'collection' list is non-empty
        if "collection" in data:
            # Extract relevant information for each summary
            for summary in data["collection"]:
                try:
                    station_info = {
                        "dateTime": summary["dateTime"],
                        "airTemperature": summary["airTemperature"],
                        "relativeHumidity": summary["relativeHumidity"],
                        "soilTemperature": summary["soilTemperature"],
                        "solarIrradiance": summary["solarIrradiance"],
                        "rainfall": summary["rainfall"],
                        "dewPoint": summary["dewPoint"],
                        "deltaT": summary["deltaT"],
                        "wetBulb": summary["wetBulb"],
                        "wind_avg_speed": summary["wind"][0]["avg"]["speed"],
                        "wind_direction": summary["wind"][0]["avg"]["direction"]["compassPoint"],
                        "barometricPressure": summary["barometricPressure"],
                        "batteryVoltage": summary["batteryVoltage"],
                    }
                except KeyError as e:
                    # Handle specific exception (e.g., key not found in the response)
                    # Use a default station_info with all values as np.nan
                    station_info = {
                        "dateTime": np.nan,
                        "airTemperature": np.nan,
                        "relativeHumidity": np.nan,
                        "soilTemperature": np.nan,
                        "solarIrradiance": np.nan,
                        "rainfall": np.nan,
                        "dewPoint": np.nan,
                        "deltaT": np.nan,
                        "wetBulb": np.nan,
                        "wind_avg_speed": np.nan,
                        "wind_direction": np.nan,
                        "barometricPressure": np.nan,
                        "batteryVoltage": np.nan,
                    }

                # Append the information for the current station and summary to the list
                all_stations_data.append(station_info)

In [5]:
# Assuming all_stations_data is a list of dictionaries
all_stations = pd.DataFrame(all_stations_data)

# Display dataframe
all_stations

,dateTime,airTemperature,relativeHumidity,soilTemperature,solarIrradiance,rainfall,dewPoint,deltaT,wetBulb,wind_avg_speed,wind_direction,barometricPressure,batteryVoltage
0,2023-07-01T00:00:00Z,6.9,72.6,7.2,24,0.0,2.3,1.8,5.1,2.66,NNE,None,12.53
1,2023-07-01T00:01:00Z,6.9,72.2,7.2,25,0.0,2.3,1.8,5.1,0.65,N,None,12.50
2,2023-07-01T00:02:00Z,6.8,72.2,7.2,25,0.0,2.2,1.8,5.0,1.15,WSW,None,12.58
3,2023-07-01T00:03:00Z,6.9,72.5,7.2,25,0.0,2.3,1.8,5.1,3.67,ENE,None,12.59
4,2023-07-01T00:04:00Z,7.0,72.3,7.2,26,0.0,2.4,1.8,5.2,1.15,ENE,None,12.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
264956,2023-12-31T23:56:00Z,21.7,54.5,26.6,479,0.0,12.1,5.2,16.5,1.66,E,None,13.74
264957,2023-12-31T23:57:00Z,21.6,54.5,26.6,484,0.0,12.0,5.2,16.4,5.18,SE,None,14.09
264958,2023-12-31T23:58:00Z,21.5,55.4,26.6,488,0.0,12.2,5.0,16.5,4.68,SSE,None,14.13
264959,2023-12-31T23:59:00Z,21.4,55.8,26.6,492,0.0,12.2,5.0,16.4,4.18,SSE,None,14.16


In [6]:
all_stations.to_csv("../Resources/raindata2.csv", index=False)